In [2]:
import numpy as np
import pandas as pd

In [3]:
data_train = pd.read_csv('/content/drive/My Drive/Train.csv')

In [4]:
data_train

,review,label
0,mature intelligent and highly charged melodram...,pos
1,http://video.google.com/videoplay?docid=211772...,pos
2,Title: Opera (1987) Director: Dario Argento Ca...,pos
3,I think a lot of people just wrote this off as...,pos
4,This is a story of two dogs and a cat looking ...,pos
...,...,...
39995,There are similarities between Ray Lawrence's ...,neg
39996,"For starters, I once met the director when he ...",neg
39997,"Much of ""Over Her Dead Body"" is so painfully u...",neg
39998,"""Lifeforce"" is a truly bizarre adaptation of t...",pos


In [17]:
x = data_train.iloc[:,0].tolist()
y = data_train.iloc[:,1].tolist()

In [6]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import sys
nltk.download('stopwords')
tokenizer = RegexpTokenizer(r'\w+')
en_stopwords = set(stopwords.words('english'))
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def getStemmedReview(review):
  review = review.lower()
  review = review.replace('<br /><br />',' ')
  #tokenization
  tokens = tokenizer.tokenize(review)
  #stopwords removal
  new_tokens = [token for token in tokens if token not in en_stopwords]
  #stemming
  stemmed_tokens = [ps.stem(token) for token in new_tokens]
  cleaned_review = ' '.join(stemmed_tokens)
  return cleaned_review

In [8]:
x_clean = [getStemmedReview(i) for i in x]

In [9]:
x_clean[:5]

['matur intellig highli charg melodrama unbelivebl film china 1948 wei wei stun perform catylast love triangl simpli stun oppurun see magnific film take',
 'http video googl com videoplay docid 211772166650071408 hl en distribut tri opt mass appeal want best possibl view rang forgo profit continu manual labor job gladli entertain work view texa tale pleas write like like alex like stuie texa texa tale write opinion rule',
 'titl opera 1987 director dario argento cast cristina masillach ian charleson urbano barberini daria nicolodi review argento movi seen suspiria one blew away style color spooki stori line next decid go opera told one best man think discov ultim one favorit horror director opera young opera singer get big break main star creepi modern opera take mc beth get hit car betti understudi get part bad psycho make watch brutal murder friend co worker wow id heard good thing flick prepar level great film would take yeah movi shortcom ill get later part movi blew away first mov

In [13]:
data_test = pd.read_csv('https://raw.githubusercontent.com/kislay960/Data-Science-Practice/master/Test.csv')

In [15]:
xtest = data_test.iloc[:,0].tolist()

In [16]:
xtest_clean = [getStemmedReview(i) for i in xtest]

In [18]:
xtest_clean[:5]

['rememb old kung fu movi use watch friday saturday late night babysitt thought charg well movi play exactli like one movi patsi kensit biggest claim fame love interest mel gibson charact lethal weapon 2 perform one reason never made big terribl actress lethal weapon 2 thought cute cute enough check movi includ love music love danc anoth big let obvious impress either attract eye soul scream turn play anoth cheap predict role done badli movi kensit star comedienn good one either work club franc cut homeland make ear bleed luck even wors french govern want throw expir visa mayb caught act get marri casanova freiss luck predict begin terribl way give movi neg rate 1 10 star rate',
 'movi anoth one list movi bother saw 40 year ago adolesc stay late annoy find 95 romanc 4 everyth els 1 histori call bait switch movi one interest titl actual movi scam subject deserv good cinemat treatment movi almost insult serv actual member lafayett escadril run law product abus home realiti idealist want 

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
cv = CountVectorizer(max_features=5000)
x_vec = cv.fit_transform(x_clean).toarray()

In [12]:
x_vec.shape

(40000, 5000)

In [19]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y_train = lb.fit_transform(y)

In [21]:
y_train

array([1, 1, 1, ..., 0, 1, 1])

In [22]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

In [23]:
mnb = MultinomialNB()

In [25]:
mnb.fit(x_vec,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [28]:
mnb.score(x_vec,y_train)

0.85285

In [29]:
bnb = BernoulliNB()

In [30]:
bnb.fit(x_vec,y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [31]:
bnb.score(x_vec,y_train)

0.854775

In [35]:
xtest_clean_vec = cv.transform(xtest_clean).toarray()
ans = bnb.predict(xtest_clean_vec)

In [44]:
ans = ['pos' if i==1 else 'neg' for i in ans]

In [47]:
ans = np.array(ans)

In [48]:
ans.shape

(10000,)

In [49]:
out = pd.DataFrame(np.hstack((data_test,ans.reshape((-1,1)))))
out.columns = ['Id','label']

In [50]:
out.head()

,Id,label
0,Remember those old kung fu movies we used to w...,neg
1,This movie is another one on my List of Movies...,pos
2,How in the world does a thing like this get in...,neg
3,"""Queen of the Damned"" is one of the best vampi...",pos
4,The Caprica episode (S01E01) is well done as a...,pos


In [52]:
from google.colab import files
out.to_csv('Sample_submission.csv',index=False) 
files.download('Sample_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>